In [19]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


In [20]:
import pandas as pd
import pymysql.cursors

In [160]:
df = pd.read_csv('dados_jogadores-1.csv', sep = ';', encoding='latin-1')
df[:10]

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,Uruguai,1.0,54.0,55.0
5,Jogador 6,Masculino,04/11/1977,Xadrez,Brasil,88.0,0.0,88.0
6,Jogador 7,Masculino,11/11/1986,Xadrez,Argentina,53.0,24.0,77.0
7,Jogador 8,Feminino,18/03/1996,Jogo da velha,Brasil,35.0,27.0,62.0
8,Jogador 9,Masculino,14/01/1969,Dama,Paraguai,27.0,18.0,45.0
9,Jogador 10,Masculino,06/05/2004,Xadrez,Chile,4.0,1.0,5.0


In [6]:
df_pais = list(df.pais.unique())
df_pais

['Chile', 'Brasil', 'Argentina', 'Uruguai', 'Paraguai']

In [7]:
df_jogo = list(df.jogo.unique())
df_jogo

['Dama', 'Jogo da velha', 'Xadrez']

In [9]:
df_jogador = df[['jogador','genero', 'data_nascimento', 'num_vitorias','num_derrotas','total_partidas']]
df_jogador[:10]

,jogador,genero,data_nascimento,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,1.0,54.0,55.0
5,Jogador 6,Masculino,04/11/1977,88.0,0.0,88.0
6,Jogador 7,Masculino,11/11/1986,53.0,24.0,77.0
7,Jogador 8,Feminino,18/03/1996,35.0,27.0,62.0
8,Jogador 9,Masculino,14/01/1969,27.0,18.0,45.0
9,Jogador 10,Masculino,06/05/2004,4.0,1.0,5.0


In [110]:
def conexao():
    return pymysql.connect(host='localhost',
                    user='root',
                    password='123',
                    database='desafio_m2',
                   cursorclass=pymysql.cursors.DictCursor
                   )
conn = conexao()
    
def inserir(table, values):
    queryInicial = f"INSERT INTO {table} VALUES "
    queryParameter = ''
    for value in values[:1]:
        parameter = "NULL, "
        for item in value:
            parameter += "%s, "
        queryParameter += "(" + parameter[:-2] + ')'

    query = queryInicial + queryParameter
    connection  = conexao()
    with connection:
        with connection.cursor() as cursor:
            cursor.executemany(query, values)
        connection.commit()
        
def consulta(table):
    sql = f"SELECT * FROM `{table}`"
    return pd.read_sql(sql, conn)

In [105]:
paises = []
for pais in df_pais:
    paises.append( [pais] )

try:
    inserir('pais',paises)
    print(f"Registro inserido com sucesso: ")
except Exception as e:
        print(f"Não foi possível inserir o registro . O erro encontrado foi: {e}") 

Registro inserido com sucesso: 


In [106]:
jogos = []
for jogo in df_jogo:
    jogos.append( [jogo] )

try:
    inserir('jogo',jogos)
    print(f"Registro inserido com sucesso: ")
except Exception as e:
        print(f"Não foi possível inserir o registro . O erro encontrado foi: {e}") 

Registro inserido com sucesso: 


In [112]:
listaPaises = consulta('pais')
listaPaises

,cod_pais,nome_pais
0,1,Chile
1,2,Brasil
2,3,Argentina
3,4,Uruguai
4,5,Paraguai


In [113]:
listaJogos = consulta('jogo')
listaJogos

C:\Users\Brito\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cod_jogo,nome_jogo
0,1,Dama
1,2,Jogo da velha
2,3,Xadrez


In [116]:
dict_pais = listaPaises.set_index(listaPaises.nome_pais)['cod_pais'].to_dict()
dict_pais


{'Chile': 1, 'Brasil': 2, 'Argentina': 3, 'Uruguai': 4, 'Paraguai': 5}

In [118]:
dict_jogo = listaJogos.set_index( listaJogos.nome_jogo )['cod_jogo'].to_dict()
dict_jogo

{'Dama': 1, 'Jogo da velha': 2, 'Xadrez': 3}

In [163]:
df.pais = df.pais.map(dict_pais)
df[:10]

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,1,1,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,2,1,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,2,2,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,2,3,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,1,4,1.0,54.0,55.0
5,Jogador 6,Masculino,04/11/1977,3,2,88.0,0.0,88.0
6,Jogador 7,Masculino,11/11/1986,3,3,53.0,24.0,77.0
7,Jogador 8,Feminino,18/03/1996,2,2,35.0,27.0,62.0
8,Jogador 9,Masculino,14/01/1969,1,5,27.0,18.0,45.0
9,Jogador 10,Masculino,06/05/2004,3,1,4.0,1.0,5.0


In [162]:
df.jogo = df.jogo.map(dict_jogo)
df[:10]



,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,1,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,2,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,2,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,2,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,1,Uruguai,1.0,54.0,55.0
5,Jogador 6,Masculino,04/11/1977,3,Brasil,88.0,0.0,88.0
6,Jogador 7,Masculino,11/11/1986,3,Argentina,53.0,24.0,77.0
7,Jogador 8,Feminino,18/03/1996,2,Brasil,35.0,27.0,62.0
8,Jogador 9,Masculino,14/01/1969,1,Paraguai,27.0,18.0,45.0
9,Jogador 10,Masculino,06/05/2004,3,Chile,4.0,1.0,5.0


## Preenchendo NaN com Zero

In [191]:
df1  =df.fillna(0)


In [192]:
jogadores = []

for jogador in df1.itertuples():   
    dt = jogador.data_nascimento.split("/")
    dataNascimento = f'{dt[2]}-{dt[1]}-{dt[0]}'
    jogadores.append(
            [
                jogador.jogador, 
                jogador.genero, 
                dataNascimento, 
                jogador.num_vitorias,
                jogador.num_derrotas,
                jogador.total_partidas,
                jogador.pais,
                jogador.jogo
            ])
jogadores[:10]

[['jogador 1', 'Masculino', '1990-09-03', 14.0, 0.0, 14.0, 1, 1],
 ['Jogador 2', 'Feminino', '1986-05-01', 39.0, 20.0, 59.0, 1, 2],
 ['Jogador 3', 'Masculino', '1964-10-04', 26.0, 3.0, 29.0, 2, 2],
 ['Jogador 4', 'Masculino', '1979-09-28', 9.0, 72.0, 81.0, 3, 2],
 ['Jogador 5', 'Masculino', '1978-09-03', 1.0, 54.0, 55.0, 4, 1],
 ['Jogador 6', 'Masculino', '1977-11-04', 88.0, 0.0, 88.0, 2, 3],
 ['Jogador 7', 'Masculino', '1986-11-11', 53.0, 24.0, 77.0, 3, 3],
 ['Jogador 8', 'Feminino', '1996-03-18', 35.0, 27.0, 62.0, 2, 2],
 ['Jogador 9', 'Masculino', '1969-01-14', 27.0, 18.0, 45.0, 5, 1],
 ['Jogador 10', 'Masculino', '2004-05-06', 4.0, 1.0, 5.0, 1, 3]]

In [193]:
try:
    inserir('jogador',jogadores)
    print(f"Registro inserido com sucesso: ")
except Exception as e:
        print(f"Não foi possível inserir o registro . O erro encontrado foi: {e}") 

Registro inserido com sucesso: 


,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,1,1,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,2,1,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,2,2,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,2,3,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,1,4,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
494,Jogador 495,Feminino,03/06/1974,1,1,52.0,28.0,80.0
495,Jogador 496,Masculino,09/02/1959,1,1,13.0,51.0,64.0
496,Jogador 497,Masculino,06/01/1962,2,3,57.0,NaN,80.0
497,Jogador 498,Masculino,28/09/1958,2,1,5.0,3.0,8.0
